In [65]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.command import Command
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import os

In [66]:
DOWNLOAD_PATH = './html/'
CAT_URL = ('https://www.g2.com/categories')

In [120]:
# Set the default download folder
chrome_driver = os.path.abspath("c:/works/utils/chromedriver/chromedriver.exe")
os.environ["webdriver.chrome.driver"] = chrome_driver
options = webdriver.ChromeOptions()
options.add_argument("--disable-extensions")
#options.add_argument("--disable-gpu")
#options.add_argument("--no-sandbox")  # linux only
#options.add_argument("--headless")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--window-size=1280,1024")
default_download_path = {"download.default_directory": DOWNLOAD_PATH}
options.add_experimental_option("prefs", default_download_path)
browser = webdriver.Chrome(executable_path=chrome_driver,
                           options=options)
browser.implicitly_wait(15)
browser.get(CAT_URL)

C:\Users\suriy\AppData\Local\Temp/ipykernel_30164/3075945727.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver,


In [68]:
xpath = "//input[contains(@class, 'js-cookie-consent-action')]"
elem = browser.find_element(By.XPATH, xpath)
elem

<selenium.webdriver.remote.webelement.WebElement (session="84e7e9bd443fe78203f0cabd00ddc7f8", element="1b7ce3b2-0074-41e0-8881-cf4ca9e8c2aa")>

In [69]:
elem.click()

In [70]:
xpath = "//div[contains(@class, 'categories__name')]"
elems = browser.find_elements(By.XPATH, xpath)
len(elems)

2057

In [71]:
elems[0].text

'CRM Software\nSales Software'

In [72]:
elems[14].text

''

In [73]:
elems[14].get_attribute('innerHTML')

'<span class="l5 mb-0"><a class="link js-log-click" data-event-options="{&quot;category_id&quot;:26,&quot;category&quot;:&quot;CPQ&quot;,&quot;name&quot;:&quot;Event::Categories::Index::CategoryVisited&quot;}" href="/categories/cpq">CPQ Software</a></span><div class="categories__parent">Quote Management Software</div>'

In [74]:
e = elems[14].find_element(By.XPATH, './div[@class="categories__parent"]')
e.get_attribute('innerHTML')

'Quote Management Software'

In [75]:
e = elems[14].find_element(By.XPATH, './/a[contains(@class, "link")]')
e.get_attribute('innerHTML')

'CPQ Software'

In [76]:
e.get_attribute('href')

'https://www.g2.com/categories/cpq'

In [93]:
cats = []
for i, e in enumerate(elems):
    text = e.text
    e2 = e.find_element(By.XPATH, './/a[contains(@class, "link")]')
    href = e2.get_attribute('href')
    cat = text.split('\n')[0]
    parent = text.split('\n')[-1]
    #text = '%s (%s) ==> %s' % (cat, parent, href)
    if e.text == '':
        e2 = e.find_element(By.XPATH, './/a[contains(@class, "link")]')
        cat = e2.get_attribute('innerHTML')
        href = e2.get_attribute('href')
        e3 = e.find_element(By.XPATH, './div[@class="categories__parent"]')
        parent = e3.get_attribute('innerHTML')
        #text = ' - %s (%s) ==> %s' % (cat, parent, href) 
    cats.append({'name': cat, 'parent': parent, 'href': href})
    if i % 100 == 0:
        print(i)
    #break
print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2056


In [94]:
df = pd.DataFrame(cats)
df

,name,parent,href
0,CRM Software,Sales Software,https://www.g2.com/categories/crm
1,Sales Compensation Software,Sales Software,https://www.g2.com/categories/sales-compensation
2,AI Sales Assistant Software,Sales Software,https://www.g2.com/categories/ai-sales-assistant
3,Auto Dialer Software,Sales Software,https://www.g2.com/categories/auto-dialer
4,Contract Analytics Software,Sales Software,https://www.g2.com/categories/contract-analytics
...,...,...,...
2052,Vineyard Management Software,Winery Software,https://www.g2.com/categories/vineyard-management
2053,Winery Management Software,Winery Software,https://www.g2.com/categories/winery-management
2054,Indoor Access Points,Wireless Hardware,https://www.g2.com/categories/indoor-access-po...
2055,Outdoor Access Points,Wireless Hardware,https://www.g2.com/categories/outdoor-access-p...


In [100]:
len(df.name.unique()), len(df.parent.unique()), len(df.href.unique())

(2057, 237, 2057)

In [102]:
df['n_sub'] = 0

In [104]:
for n in df.name.unique():
    nsub = len(df[df.parent == n])
    df.loc[df.name == n, 'n_sub'] = nsub

In [123]:
df.to_csv('g2_categories.csv', index=False)

In [109]:
sdf = df[df.n_sub==0].copy()
sdf

,name,parent,href,n_sub
0,CRM Software,Sales Software,https://www.g2.com/categories/crm,0
1,Sales Compensation Software,Sales Software,https://www.g2.com/categories/sales-compensation,0
2,AI Sales Assistant Software,Sales Software,https://www.g2.com/categories/ai-sales-assistant,0
3,Auto Dialer Software,Sales Software,https://www.g2.com/categories/auto-dialer,0
4,Contract Analytics Software,Sales Software,https://www.g2.com/categories/contract-analytics,0
...,...,...,...,...
2052,Vineyard Management Software,Winery Software,https://www.g2.com/categories/vineyard-management,0
2053,Winery Management Software,Winery Software,https://www.g2.com/categories/winery-management,0
2054,Indoor Access Points,Wireless Hardware,https://www.g2.com/categories/indoor-access-po...,0
2055,Outdoor Access Points,Wireless Hardware,https://www.g2.com/categories/outdoor-access-p...,0


In [121]:
for i, r in sdf.iterrows():
    print(r)
    browser.get(r.href)
    break

name                           CRM Software
parent                       Sales Software
href      https://www.g2.com/categories/crm
n_sub                                     0
Name: 0, dtype: object


In [122]:
browser.quit()